In [2]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
import tensorflow as tf
import seaborn as sns
import sklearn
import random
import math
import time


from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from tensorflow.keras import optimizers


from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
from methods import *

%matplotlib inline
warnings.filterwarnings('ignore')

SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

In [3]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD004.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR', 'farB','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 120
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(61249, 27) (41214, 26) (248, 1)


In [16]:
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(LSTM(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    # model.save_weights(weights_file)

    return model

In [17]:
%%time
results_all = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.3
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all = pd.concat([results_all, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all.to_csv('results/all/fd004.csv')

(51538, 40, 17) (51538, 1) (248, 40, 17)
Epoch 1/20
1611/1611 [==============================] - 13s 7ms/step - loss: 762.7661 - val_loss: 328.0673
Epoch 2/20
1611/1611 [==============================] - 11s 7ms/step - loss: 294.9990 - val_loss: 232.8337
Epoch 3/20
1611/1611 [==============================] - 11s 7ms/step - loss: 249.8761 - val_loss: 213.0574
Epoch 4/20
1611/1611 [==============================] - 12s 7ms/step - loss: 231.6885 - val_loss: 207.1930
Epoch 5/20
1611/1611 [==============================] - 11s 7ms/step - loss: 215.2335 - val_loss: 189.2900
Epoch 6/20
1611/1611 [==============================] - 11s 7ms/step - loss: 200.3619 - val_loss: 187.0126
Epoch 7/20
1611/1611 [==============================] - 12s 7ms/step - loss: 189.2251 - val_loss: 180.8434
Epoch 8/20
1611/1611 [==============================] - 11s 7ms/step - loss: 182.3082 - val_loss: 169.7781
Epoch 9/20
1611/1611 [==============================] - 11s 7ms/step - loss: 175.0479 - val_loss: 163.9

(51538, 40, 17) (51538, 1) (248, 40, 17)
Epoch 1/20
1611/1611 [==============================] - 14s 8ms/step - loss: 768.5524 - val_loss: 357.2986
Epoch 2/20
1611/1611 [==============================] - 11s 7ms/step - loss: 307.8346 - val_loss: 258.7034
Epoch 3/20
1611/1611 [==============================] - 11s 7ms/step - loss: 260.2329 - val_loss: 259.2318
Epoch 4/20
1611/1611 [==============================] - 11s 7ms/step - loss: 236.9283 - val_loss: 207.3871
Epoch 5/20
1611/1611 [==============================] - 11s 7ms/step - loss: 221.7244 - val_loss: 203.9132
Epoch 6/20
1611/1611 [==============================] - 11s 7ms/step - loss: 206.4735 - val_loss: 193.6375
Epoch 7/20
1611/1611 [==============================] - 11s 7ms/step - loss: 194.2180 - val_loss: 170.4289
Epoch 8/20
1611/1611 [==============================] - 11s 7ms/step - loss: 182.5528 - val_loss: 178.5842
Epoch 9/20
1611/1611 [==============================] - 11s 7ms/step - loss: 176.8829 - val_loss: 168.9

(51538, 40, 17) (51538, 1) (248, 40, 17)
Epoch 1/20
1611/1611 [==============================] - 13s 8ms/step - loss: 753.3668 - val_loss: 332.4726
Epoch 2/20
1611/1611 [==============================] - 11s 7ms/step - loss: 291.6102 - val_loss: 275.1886
Epoch 3/20
1611/1611 [==============================] - 11s 7ms/step - loss: 256.3381 - val_loss: 245.3007
Epoch 4/20
1611/1611 [==============================] - 11s 7ms/step - loss: 244.8977 - val_loss: 252.5877
Epoch 5/20
1611/1611 [==============================] - 11s 7ms/step - loss: 226.0944 - val_loss: 216.1996
Epoch 6/20
1611/1611 [==============================] - 11s 7ms/step - loss: 215.0120 - val_loss: 199.5721
Epoch 7/20
1611/1611 [==============================] - 11s 7ms/step - loss: 203.7311 - val_loss: 202.4122
Epoch 8/20
1611/1611 [==============================] - 11s 7ms/step - loss: 193.6084 - val_loss: 189.6073
Epoch 9/20
1611/1611 [==============================] - 11s 7ms/step - loss: 184.7121 - val_loss: 191.5

(51538, 40, 17) (51538, 1) (248, 40, 17)
Epoch 1/20
1611/1611 [==============================] - 13s 8ms/step - loss: 817.0692 - val_loss: 340.1449
Epoch 2/20
1611/1611 [==============================] - 11s 7ms/step - loss: 295.0835 - val_loss: 244.6979
Epoch 3/20
1611/1611 [==============================] - 13s 8ms/step - loss: 255.6130 - val_loss: 248.2810
Epoch 4/20
1611/1611 [==============================] - 12s 7ms/step - loss: 235.3248 - val_loss: 237.1968
Epoch 5/20
1611/1611 [==============================] - 11s 7ms/step - loss: 223.5804 - val_loss: 238.4308
Epoch 6/20
1611/1611 [==============================] - 11s 7ms/step - loss: 206.3964 - val_loss: 202.0321
Epoch 7/20
1611/1611 [==============================] - 11s 7ms/step - loss: 196.3786 - val_loss: 190.3189
Epoch 8/20
1611/1611 [==============================] - 11s 7ms/step - loss: 187.0319 - val_loss: 225.8476
Epoch 9/20
1611/1611 [==============================] - 12s 7ms/step - loss: 179.6189 - val_loss: 172.6

(51538, 40, 17) (51538, 1) (248, 40, 17)
Epoch 1/20
1611/1611 [==============================] - 13s 8ms/step - loss: 866.2994 - val_loss: 383.3265
Epoch 2/20
1611/1611 [==============================] - 11s 7ms/step - loss: 305.2556 - val_loss: 270.8068
Epoch 3/20
1611/1611 [==============================] - 11s 7ms/step - loss: 249.9944 - val_loss: 224.3305
Epoch 4/20
1611/1611 [==============================] - 11s 7ms/step - loss: 229.3898 - val_loss: 264.2542
Epoch 5/20
1611/1611 [==============================] - 11s 7ms/step - loss: 210.1374 - val_loss: 216.4985
Epoch 6/20
1611/1611 [==============================] - 11s 7ms/step - loss: 198.5177 - val_loss: 236.7857
Epoch 7/20
1611/1611 [==============================] - 11s 7ms/step - loss: 189.3521 - val_loss: 207.5854
Epoch 8/20
1611/1611 [==============================] - 11s 7ms/step - loss: 182.1673 - val_loss: 218.2080
Epoch 9/20
1611/1611 [==============================] - 13s 8ms/step - loss: 176.9418 - val_loss: 232.2

In [18]:
results_all

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,12.551669,0.0,741.853794,0.0,157.544388,0.0,[64],0.2,tanh,32,40,220.229309
1,12.369032,0.0,720.105677,0.0,152.992905,0.0,[64],0.2,tanh,32,40,227.559174
2,12.738909,0.0,804.189686,0.0,162.279831,0.0,[64],0.2,tanh,32,40,228.795339
3,13.329341,0.0,641.524037,0.0,177.671310,0.0,[64],0.2,tanh,32,40,230.277804
4,13.816006,0.0,775.655890,0.0,190.882004,0.0,[64],0.2,tanh,32,40,230.477888


### FD002 <a class="anchor" id="fd002"></a>

In [19]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [20]:
%%time
results_all002 = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.2
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [32]
    dropout = 0.1
    activation = 'tanh'
    batch_size = 128
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all002 = pd.concat([results_all002, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all002.to_csv('results/all/fd002.csv')

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 4s 9ms/step - loss: 2284.7837 - val_loss: 1184.4233
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 488.9340 - val_loss: 232.4040
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 277.7340 - val_loss: 208.3209
Epoch 4/20
341/341 [==============================] - 3s 9ms/step - loss: 252.8589 - val_loss: 210.1246
Epoch 5/20
341/341 [==============================] - 3s 9ms/step - loss: 239.3640 - val_loss: 197.7523
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 235.1574 - val_loss: 202.5185
Epoch 7/20
341/341 [==============================] - 3s 8ms/step - loss: 233.2388 - val_loss: 186.3883
Epoch 8/20
341/341 [==============================] - 3s 8ms/step - loss: 230.1901 - val_loss: 191.8869
Epoch 9/20
341/341 [==============================] - 3s 9ms/step - loss: 224.1405 - val_loss: 196.1385
Epoch 10/20
341/341 [

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 4s 9ms/step - loss: 2342.7776 - val_loss: 1485.9114
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 501.3373 - val_loss: 232.1009
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 284.6033 - val_loss: 210.2034
Epoch 4/20
341/341 [==============================] - 3s 9ms/step - loss: 268.0725 - val_loss: 190.1271
Epoch 5/20
341/341 [==============================] - 3s 9ms/step - loss: 245.6246 - val_loss: 201.6791
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 242.2276 - val_loss: 184.8576
Epoch 7/20
341/341 [==============================] - 3s 9ms/step - loss: 235.3008 - val_loss: 181.2748
Epoch 8/20
341/341 [==============================] - 3s 9ms/step - loss: 233.1525 - val_loss: 184.5342
Epoch 9/20
341/341 [==============================] - 3s 9ms/step - loss: 228.3228 - val_loss: 183.7434
Epoch 10/20
341/341 [

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 4s 9ms/step - loss: 2312.6143 - val_loss: 1375.4172
Epoch 2/20
341/341 [==============================] - 3s 8ms/step - loss: 517.6144 - val_loss: 241.2267
Epoch 3/20
341/341 [==============================] - 3s 8ms/step - loss: 293.7717 - val_loss: 206.6121
Epoch 4/20
341/341 [==============================] - 3s 8ms/step - loss: 264.7542 - val_loss: 202.7057
Epoch 5/20
341/341 [==============================] - 3s 8ms/step - loss: 256.7372 - val_loss: 196.3154
Epoch 6/20
341/341 [==============================] - 3s 8ms/step - loss: 245.3320 - val_loss: 191.3672
Epoch 7/20
341/341 [==============================] - 3s 9ms/step - loss: 241.0967 - val_loss: 199.1064
Epoch 8/20
341/341 [==============================] - 3s 9ms/step - loss: 235.3632 - val_loss: 200.3324
Epoch 9/20
341/341 [==============================] - 3s 9ms/step - loss: 230.2278 - val_loss: 190.0517
Epoch 10/20
341/341 [

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 4s 10ms/step - loss: 2347.4243 - val_loss: 1551.2659
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 551.0797 - val_loss: 245.1803
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 275.3705 - val_loss: 195.8227
Epoch 4/20
341/341 [==============================] - 3s 9ms/step - loss: 253.0560 - val_loss: 224.1236
Epoch 5/20
341/341 [==============================] - 3s 9ms/step - loss: 243.1407 - val_loss: 183.4476
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 232.0502 - val_loss: 181.7664
Epoch 7/20
341/341 [==============================] - 3s 9ms/step - loss: 228.7188 - val_loss: 182.9298
Epoch 8/20
341/341 [==============================] - 3s 9ms/step - loss: 228.1400 - val_loss: 176.8402
Epoch 9/20
341/341 [==============================] - 3s 9ms/step - loss: 224.3821 - val_loss: 185.5224
Epoch 10/20
341/341 

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 4s 9ms/step - loss: 2275.6174 - val_loss: 1325.1198
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 536.1032 - val_loss: 243.3846
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 279.8450 - val_loss: 202.9051
Epoch 4/20
341/341 [==============================] - 3s 9ms/step - loss: 252.0998 - val_loss: 207.0529
Epoch 5/20
341/341 [==============================] - 3s 8ms/step - loss: 242.0358 - val_loss: 183.7728
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 231.5358 - val_loss: 178.5688
Epoch 7/20
341/341 [==============================] - 3s 8ms/step - loss: 228.2318 - val_loss: 203.5125
Epoch 8/20
341/341 [==============================] - 3s 8ms/step - loss: 226.3349 - val_loss: 173.9748
Epoch 9/20
341/341 [==============================] - 3s 9ms/step - loss: 221.1027 - val_loss: 197.2489
Epoch 10/20
341/341 [

In [21]:
results_all002

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,13.386268,0.0,933.507017,0.0,179.192200,0.0,[32],0.1,tanh,128,40,60.836705
1,13.349602,0.0,974.155848,0.0,178.211899,0.0,[32],0.1,tanh,128,40,59.740708
2,13.347993,0.0,934.270798,0.0,178.168915,0.0,[32],0.1,tanh,128,40,58.691479
3,13.549702,0.0,900.602838,0.0,183.594406,0.0,[32],0.1,tanh,128,40,64.237170
4,13.642462,0.0,867.849042,0.0,186.116791,0.0,[32],0.1,tanh,128,40,58.900007


### FD003 <a class="anchor" id="fd003"></a>

In [22]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [23]:
%%time
results_all003 = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.1
    sequence_length = 35
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all003 = pd.concat([results_all003, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all003.to_csv('results/all/fd003.csv')

(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 9s 6ms/step - loss: 886.5042 - val_loss: 319.7583
Epoch 2/20
1404/1404 [==============================] - 8s 6ms/step - loss: 283.5926 - val_loss: 230.9650
Epoch 3/20
1404/1404 [==============================] - 8s 6ms/step - loss: 252.2755 - val_loss: 201.6743
Epoch 4/20
1404/1404 [==============================] - 8s 6ms/step - loss: 237.7799 - val_loss: 189.3466
Epoch 5/20
1404/1404 [==============================] - 8s 6ms/step - loss: 227.3036 - val_loss: 171.3220
Epoch 6/20
1404/1404 [==============================] - 9s 6ms/step - loss: 218.2708 - val_loss: 168.0999
Epoch 7/20
1404/1404 [==============================] - 9s 7ms/step - loss: 214.3134 - val_loss: 181.3601
Epoch 8/20
1404/1404 [==============================] - 9s 6ms/step - loss: 206.2580 - val_loss: 177.7361
Epoch 9/20
1404/1404 [==============================] - 9s 6ms/step - loss: 199.8463 - val_loss: 170.3373
Epoch

(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 10s 6ms/step - loss: 772.4301 - val_loss: 205.7240
Epoch 2/20
1404/1404 [==============================] - 9s 6ms/step - loss: 272.9133 - val_loss: 185.1960
Epoch 3/20
1404/1404 [==============================] - 9s 6ms/step - loss: 243.5848 - val_loss: 180.9420
Epoch 4/20
1404/1404 [==============================] - 9s 6ms/step - loss: 233.5709 - val_loss: 199.6880
Epoch 5/20
1404/1404 [==============================] - 9s 6ms/step - loss: 222.2003 - val_loss: 177.5709
Epoch 6/20
1404/1404 [==============================] - 10s 7ms/step - loss: 214.3226 - val_loss: 180.2964
Epoch 7/20
1404/1404 [==============================] - 9s 7ms/step - loss: 207.2195 - val_loss: 185.6786
Epoch 8/20
1404/1404 [==============================] - 10s 7ms/step - loss: 201.4224 - val_loss: 175.3193
Epoch 9/20
1404/1404 [==============================] - 9s 7ms/step - loss: 199.0803 - val_loss: 171.6209
Ep

(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 12s 7ms/step - loss: 888.2208 - val_loss: 256.3362
Epoch 2/20
1404/1404 [==============================] - 10s 7ms/step - loss: 272.3698 - val_loss: 206.1081
Epoch 3/20
1404/1404 [==============================] - 9s 6ms/step - loss: 244.7237 - val_loss: 182.8092
Epoch 4/20
1404/1404 [==============================] - 9s 6ms/step - loss: 229.8972 - val_loss: 220.7837
Epoch 5/20
1404/1404 [==============================] - 10s 7ms/step - loss: 221.9341 - val_loss: 183.2493
Epoch 6/20
1404/1404 [==============================] - 10s 7ms/step - loss: 213.7457 - val_loss: 189.4396
Epoch 7/20
1404/1404 [==============================] - 9s 6ms/step - loss: 209.1642 - val_loss: 180.9539
Epoch 8/20
1404/1404 [==============================] - 9s 7ms/step - loss: 202.3134 - val_loss: 191.4040
Epoch 9/20
1404/1404 [==============================] - 9s 6ms/step - loss: 197.8971 - val_loss: 170.5309
E

(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 10s 7ms/step - loss: 861.8621 - val_loss: 234.3589
Epoch 2/20
1404/1404 [==============================] - 9s 6ms/step - loss: 292.9422 - val_loss: 252.8376
Epoch 3/20
1404/1404 [==============================] - 9s 6ms/step - loss: 255.3584 - val_loss: 187.7728
Epoch 4/20
1404/1404 [==============================] - 9s 6ms/step - loss: 245.7108 - val_loss: 186.2925
Epoch 5/20
1404/1404 [==============================] - 9s 6ms/step - loss: 233.0231 - val_loss: 192.4771
Epoch 6/20
1404/1404 [==============================] - 9s 6ms/step - loss: 225.1458 - val_loss: 178.6168
Epoch 7/20
1404/1404 [==============================] - 9s 6ms/step - loss: 216.9049 - val_loss: 178.9526
Epoch 8/20
1404/1404 [==============================] - 9s 7ms/step - loss: 210.5146 - val_loss: 178.4032
Epoch 9/20
1404/1404 [==============================] - 10s 7ms/step - loss: 205.4955 - val_loss: 170.8001
Epo

(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 10s 7ms/step - loss: 793.4174 - val_loss: 299.7874
Epoch 2/20
1404/1404 [==============================] - 9s 6ms/step - loss: 270.9649 - val_loss: 195.3567
Epoch 3/20
1404/1404 [==============================] - 9s 6ms/step - loss: 246.2209 - val_loss: 182.0868
Epoch 4/20
1404/1404 [==============================] - 9s 6ms/step - loss: 233.2557 - val_loss: 223.7067
Epoch 5/20
1404/1404 [==============================] - 9s 6ms/step - loss: 221.8022 - val_loss: 193.7404
Epoch 6/20
1404/1404 [==============================] - 9s 6ms/step - loss: 213.4007 - val_loss: 174.4439
Epoch 7/20
1404/1404 [==============================] - 9s 6ms/step - loss: 207.2265 - val_loss: 175.5321
Epoch 8/20
1404/1404 [==============================] - 9s 7ms/step - loss: 201.6379 - val_loss: 163.5542
Epoch 9/20
1404/1404 [==============================] - 10s 7ms/step - loss: 195.2943 - val_loss: 177.1051
Epo

In [24]:
results_all003

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,12.369151,0.0,653.681430,0.0,152.995911,0.0,[64],0.2,tanh,32,35,176.937613
1,12.660530,0.0,708.243193,0.0,160.289001,0.0,[64],0.2,tanh,32,35,188.546215
2,13.248932,0.0,828.569532,0.0,175.534210,0.0,[64],0.2,tanh,32,35,183.073372
3,12.791377,0.0,791.082270,0.0,163.619324,0.0,[64],0.2,tanh,32,35,186.517013
4,12.889018,0.0,637.156003,0.0,166.126801,0.0,[64],0.2,tanh,32,35,181.034672


### FD001

In [25]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(20631, 27) (13096, 26) (100, 1)


In [26]:
%%time
results_all001 = pd.DataFrame()
for SEED in range(5):  
    set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.1
    sequence_length = 30
    epochs = 20
    nodes_per_layer = [128]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 64
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all001 = pd.concat([results_all001, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all001.to_csv('results/all/fd001.csv')

(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 6s 17ms/step - loss: 1945.8270 - val_loss: 1011.8328
Epoch 2/20
278/278 [==============================] - 5s 17ms/step - loss: 557.1711 - val_loss: 283.6362
Epoch 3/20
278/278 [==============================] - 5s 17ms/step - loss: 336.0125 - val_loss: 402.8049
Epoch 4/20
278/278 [==============================] - 5s 17ms/step - loss: 256.9118 - val_loss: 234.6717
Epoch 5/20
278/278 [==============================] - 5s 17ms/step - loss: 240.9247 - val_loss: 188.8959
Epoch 6/20
278/278 [==============================] - 5s 16ms/step - loss: 215.9529 - val_loss: 246.9015
Epoch 7/20
278/278 [==============================] - 5s 16ms/step - loss: 212.4774 - val_loss: 200.7240
Epoch 8/20
278/278 [==============================] - 5s 17ms/step - loss: 196.8468 - val_loss: 287.7190
Epoch 9/20
278/278 [==============================] - 5s 16ms/step - loss: 198.1949 - val_loss: 179.0840
Epoch 10/20


(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 6s 17ms/step - loss: 1952.8248 - val_loss: 989.5231
Epoch 2/20
278/278 [==============================] - 4s 16ms/step - loss: 608.6642 - val_loss: 457.7419
Epoch 3/20
278/278 [==============================] - 5s 17ms/step - loss: 358.4405 - val_loss: 682.9811
Epoch 4/20
278/278 [==============================] - 5s 16ms/step - loss: 251.7239 - val_loss: 341.1137
Epoch 5/20
278/278 [==============================] - 5s 16ms/step - loss: 211.5727 - val_loss: 217.0099
Epoch 6/20
278/278 [==============================] - 5s 16ms/step - loss: 201.9030 - val_loss: 207.4494
Epoch 7/20
278/278 [==============================] - 7s 24ms/step - loss: 192.0596 - val_loss: 207.3164
Epoch 8/20
278/278 [==============================] - 5s 16ms/step - loss: 189.4069 - val_loss: 176.4110
Epoch 9/20
278/278 [==============================] - 5s 17ms/step - loss: 183.2050 - val_loss: 192.1585
Epoch 10/20
2

(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 5s 16ms/step - loss: 1969.4290 - val_loss: 1184.3573
Epoch 2/20
278/278 [==============================] - 4s 16ms/step - loss: 569.7576 - val_loss: 422.7188
Epoch 3/20
278/278 [==============================] - 4s 16ms/step - loss: 306.3215 - val_loss: 222.5002
Epoch 4/20
278/278 [==============================] - 4s 16ms/step - loss: 235.5519 - val_loss: 214.6935
Epoch 5/20
278/278 [==============================] - 4s 16ms/step - loss: 214.0097 - val_loss: 179.1317
Epoch 6/20
278/278 [==============================] - 4s 16ms/step - loss: 201.6206 - val_loss: 170.3593
Epoch 7/20
278/278 [==============================] - 4s 16ms/step - loss: 195.8453 - val_loss: 250.8557
Epoch 8/20
278/278 [==============================] - 4s 16ms/step - loss: 191.1101 - val_loss: 202.9509
Epoch 9/20
278/278 [==============================] - 4s 16ms/step - loss: 186.9187 - val_loss: 181.4738
Epoch 10/20


(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 5s 16ms/step - loss: 1981.1896 - val_loss: 917.8284
Epoch 2/20
278/278 [==============================] - 5s 17ms/step - loss: 589.6582 - val_loss: 387.9435
Epoch 3/20
278/278 [==============================] - 5s 17ms/step - loss: 354.6008 - val_loss: 215.8253
Epoch 4/20
278/278 [==============================] - 5s 16ms/step - loss: 249.8626 - val_loss: 274.0669
Epoch 5/20
278/278 [==============================] - 4s 16ms/step - loss: 218.3911 - val_loss: 303.3423
Epoch 6/20
278/278 [==============================] - 5s 16ms/step - loss: 212.5883 - val_loss: 251.4697
Epoch 7/20
278/278 [==============================] - 5s 17ms/step - loss: 202.4792 - val_loss: 616.0234
Epoch 8/20
278/278 [==============================] - 4s 16ms/step - loss: 196.4964 - val_loss: 172.3805
Epoch 9/20
278/278 [==============================] - 4s 16ms/step - loss: 184.9444 - val_loss: 233.0498
Epoch 10/20
2

(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 5s 16ms/step - loss: 2036.7043 - val_loss: 1121.9376
Epoch 2/20
278/278 [==============================] - 4s 16ms/step - loss: 604.8963 - val_loss: 474.9707
Epoch 3/20
278/278 [==============================] - 4s 16ms/step - loss: 448.0006 - val_loss: 532.3290
Epoch 4/20
278/278 [==============================] - 4s 16ms/step - loss: 276.0841 - val_loss: 333.5952
Epoch 5/20
278/278 [==============================] - 4s 16ms/step - loss: 227.4386 - val_loss: 538.3504
Epoch 6/20
278/278 [==============================] - 5s 16ms/step - loss: 220.6311 - val_loss: 231.7316
Epoch 7/20
278/278 [==============================] - 4s 16ms/step - loss: 198.7297 - val_loss: 169.9673
Epoch 8/20
278/278 [==============================] - 4s 16ms/step - loss: 197.7237 - val_loss: 208.0873
Epoch 9/20
278/278 [==============================] - 4s 16ms/step - loss: 189.4026 - val_loss: 173.8299
Epoch 10/20
